## Ergänzungs-Notebook zur Übung 1: Embeddings mit GPU-Beschleunigung

Dieses Notebook ist eine Ergänzung zu **Übung 1** und zeigt, wie die Berechnung von Embeddings mit **GPU-Beschleunigung** statt auf einer CPU durchgeführt wird. In der ursprünglichen Übung 1 wurden die Embeddings noch mit der **CPU** berechnet, was bei größeren Datenmengen deutlich mehr Zeit in Anspruch nahm. 

Hier verwenden wir eine **GPU** (falls verfügbar), um zu demonstrieren, wie viel schneller die Berechnungen durchgeführt werden können. Dazu nutzen wir das vortrainierte Modell **`distiluse-base-multilingual-cased-v2`** aus der `sentence-transformers`-Bibliothek, um Embeddings für bis zu **50.000** Einträge aus der Tabelle `immobilien` zu berechnen.

Ziel dieses ergänzenden Notebooks ist es, die Leistungsfähigkeit der GPU beim Berechnen von Embeddings hervorzuheben und zu zeigen, wie durch den Wechsel von der CPU auf die GPU die Verarbeitungsgeschwindigkeit drastisch gesteigert werden kann.

## Vorbereitung der Umgebung

Hier werden die erforderlichen Pakete installiert und die Umgebung für die GPU-gestützte Embedding-Berechnung vorbereitet:

- **Installation**: `oracledb` (Datenbankzugriff), `sentence_transformers` (Embedding-Modell) und `torch` (GPU-Beschleunigung).
- **Initialisierung**: Import der Bibliotheken und Prüfung der GPU-Verfügbarkeit.
- **Parameter**: Verarbeitung von 50.000 Datensätzen (`numembed`).
- **Versionen**: Ausgabe der Versionen von `oracledb` und `torch`, um die korrekte Installation zu prüfen.

In [ ]:
!pip install oracledb
!pip install sentence_transformers
!pip install torch
!pip install tqdm

In [40]:
import os
import oracledb
import sys
import array
import time
import datetime
import warnings
import json
import torch
from tqdm import tqdm
warnings.filterwarnings('ignore')
from sentence_transformers import SentenceTransformer

# Anzahl Rows, die embedded werden sollen
numembed = 500

In [41]:
print("oracledb-Version:",oracledb.__version__)
print("torch-Version   :", torch.__version__)
print('Hostname : ',os.uname()[1])
print('CPU-Cores: ',os.cpu_count())
pipe = os.popen("nvidia-smi -L", "r")
print(pipe.read())
pipe.close()

oracledb-Version: 2.5.0
torch-Version   : 2.5.0+cu121
Hostname :  norbert-ai-workshop-jupyter-696b54dddf-przv4
CPU-Cores:  30
GPU 0: NVIDIA A10 (UUID: GPU-123efe7d-b688-7742-6134-04df5ae182cd)



## Datenbankverbindung herstellen

In diesem Abschnitt wird die Verbindung zur Oracle-Datenbank eingerichtet:

- **Instant Client Initialisierung**: Der Pfad zur Oracle Instant Client-Bibliothek wird angegeben.
- **Verbindungszeichenfolge**: Hostname und PDB-Name werden aus Umgebungsvariablen gelesen und zu einem Connection String (`cs`) kombiniert.
- **Datenbankverbindung**: Mit den Anmeldedaten wird eine Verbindung zur Oracle-Datenbank hergestellt und ausgegeben.

In [42]:
d = '/home/jovyan/.jupyter/instantclient_23_5'
oracledb.init_oracle_client(lib_dir=d)
host = os.environ.get('HOST_NAME')
pdb = os.environ.get('PDB_NAME')
cs = host + '/' + pdb
print(cs)

db23ai.subbb3fff175.quickcluster.oraclevcn.com/norbert.subbb3fff175.quickcluster.oraclevcn.com


In [43]:
connection = oracledb.connect(user='vector', password='vector', dsn=cs)
print(connection)

<oracledb.Connection to vector@db23ai.subbb3fff175.quickcluster.oraclevcn.com/norbert.subbb3fff175.quickcluster.oraclevcn.com>


## Auswahl der Rechenressourcen und Laden des Modells

Dieser Abschnitt wählt die Rechenressourcen aus und lädt das Embedding-Modell.

- **GPU oder CPU**: Überprüft, ob eine GPU verfügbar ist (`cuda:0`), andernfalls wird die CPU verwendet. Bei GPU-Nutzung werden die Geräteinformationen ausgegeben.
- **Modellwahl**: Das vortrainierte Modell **`distiluse-base-multilingual-cased-v2`** wird für die Berechnung der Embeddings geladen.
- **Modell auf Gerät übertragen**: Das Modell wird auf die verfügbare Rechenressource (GPU) übertragen.

In [44]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print (device)
if device == "cuda:0":
    devprop = torch.cuda.get_device_properties(0)
    print(devprop)

cuda:0
_CudaDeviceProperties(name='NVIDIA A10', major=8, minor=6, total_memory=22502MB, multi_processor_count=72, uuid=123efe7d-b688-7742-6134-04df5ae182cd, L2_cache_size=6MB)


In [45]:
embedding_model = "sentence-transformers/distiluse-base-multilingual-cased-v2"

print("Using " + embedding_model)

model = SentenceTransformer(embedding_model)
model = model.to(device)

Using sentence-transformers/distiluse-base-multilingual-cased-v2


## Berechnung der Embeddings und Update der `immobilien`-Tabelle

In diesem Abschnitt werden Embeddings für bis zu 50.000 Einträge aus der Tabelle `immobilien` auf der GPU berechnet und in der Datenbank aktualisiert. Der Prozess wird durch die parallele Verarbeitung und die Nutzung der GPU beschleunigt.

### Schritte:

1. **Datenabfrage und Trigger-Management**:
   - Es werden bis zu 50.000 Datensätze (`pid` und `beschreibung`) aus der `immobilien`-Tabelle abgefragt. Der Trigger `IMMOBILIEN_EMBED` wird deaktiviert, um während des Updates unnötige Operationen zu vermeiden.

2. **Berechnung der Embeddings**:
   - Für jeden Datensatz wird der Text aus `beschreibung` mithilfe des Modells in ein Embedding umgewandelt. Diese Embeddings werden zur späteren Verwendung in einem Array gespeichert.

3. **Aktualisierung der Datenbank**:
   - Die berechneten Embeddings werden in der `embed`-Spalte der Tabelle gespeichert. Nach dem Update wird die Transaktion in der Datenbank abgeschlossen (`commit`).

4. **Trigger-Reaktivierung**:
   - Nach dem Update wird der Trigger wieder aktiviert, um die normale Funktion der Tabelle wiederherzustellen.

In [46]:
# Embedding und anschliessendes Update der immobilien-Tabelle

curs = connection.cursor()
print("Now Vectorizing",numembed,"rows with", device)
    
sql = """select /*+ parallel(im,4) */ pid, beschreibung
         from immobilien im where rownum <="""+str(numembed)+"""order by 1"""

binds = []
tic = time.perf_counter()

# Trigger disablen wegen späterem Update 
sql1="""
BEGIN
 EXECUTE IMMEDIATE ('ALTER TRIGGER IMMOBILIEN_EMBED DISABLE');
END; """
curs.execute(sql1)

curs.execute(sql)
rows = curs.fetchall()

for id_val, description in tqdm(rows,
                                file=sys.stdout,
                                bar_format='[{elapsed}<{remaining}] {n_fmt}/{total_fmt} | {l_bar}{bar} {rate_fmt}{postfix}', colour='green'):
    data = f"[ {description} ]"
    embedding = list(model.encode(data))
    vec2 = array.array("f", embedding)
    binds.append([vec2, id_val])

toc = time.perf_counter()

seconds = toc - tic
formatted_time = time.strftime("%H:%M:%S", time.gmtime(seconds))

print(f"Embedding with",numembed,"rows on",device,f"took (h:m:s)",formatted_time)

curs.executemany(
 """update /*+ enable_parallel_dml parallel(tim,4) */ immobilien tim set embed = :1
    where pid = :2""",
    binds,
)
connection.commit()
toc2 = time.perf_counter()
print(f"Updating table took {toc2 - toc:0.4f} seconds")

# Trigger wieder enablen
sql2="""
BEGIN
 EXECUTE IMMEDIATE ('ALTER TRIGGER IMMOBILIEN_EMBED ENABLE');
END; """
curs.execute(sql2)


Now Vectorizing 500 rows with cuda:0
[00:02<00:00] 500/500 | 100%|██████████ 236.97it/s
Embedding with 500 rows on cuda:0 took (h:m:s) 00:00:02
Updating table took 0.0328 seconds


## Vorteile der GPU-Nutzung

Die GPU-Beschleunigung ermöglicht eine deutlich schnellere Berechnung von Embeddings, besonders bei großen Datensätzen. Im Vergleich zur CPU wird die Verarbeitung erheblich beschleunigt, was die Effizienz bei rechenintensiven Aufgaben steigert.